In [83]:
import pandas as pd
import re

# ЧИТАЕМ СУЩЕСТВУЩИЙ ДАТАСЕТ ДЛЯ ПРЕДОТВАРЩЕНИЯ ДУБЛИКАТОВ

In [2]:
df_fin = pd.read_csv("HYBRID_Steps12_unsafe_only.csv")

In [3]:
# df_fin = pd.concat([df1,df2,df3],0, ignore_index = True)

In [4]:
df_fin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35635 entries, 0 to 35634
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   text    35635 non-null  object 
 1   unsafe  35635 non-null  float64
dtypes: float64(1), object(1)
memory usage: 556.9+ KB


In [5]:
existing_samples_set = set(df_fin['text'])

In [1]:
def get_len(text):
    if len(text.split()) > 5 and len (text)< 250:
        return True
    return None
def is_mostly_russian(text):
    text = str(text)
    russian_letters_count = len(re.findall("[а-яА-Я]",text))
    eng_letters_count = len(re.findall("[a-zA-Z]",text))
    if russian_letters_count > eng_letters_count:
        return True
    return None
def depersonalize(text):
    text = str(text)
    url_regex = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"
    text= re.sub(url_regex, "url", text)
    text = re.sub("id[\d]*",'',text)
    text = re.sub("> ?",'',text)
    text = re.sub("@[\w]*",'',text)
#     text = re.sub("\+[\d]*", "", text)
    text = re.sub("[\d]+", "NUMBER", text)
    text = re.sub(">>", "", text)
#     text = re.sub("[\d]{3,100}", "", text)
    text = re.sub("[\t|\n|\r]", "", text)
#     text = re.sub("[(OP)|url|nickname|phone_number]", "", text)
    return text
def lemmatize(text):
    sentences_list = [s.text for s in sentenize(text)]
    tokenized_sentences = [re.sub("[^а-яА-Я ]", ' ',s).strip() for s in sentences_list]
    tokenized_sentences = [s.split() for s in tokenized_sentences]
    lemmas_list = [[morph.parse(w)[0].normal_form for w in words] for words in tokenized_sentences]
    lemmas_string = [' '.join(sent) for sent in lemmas_list]
    lemmas_string = '. '.join(lemmas_string)
    return lemmas_string

In [7]:
# df_fin['text'] = df_fin['text'].apply(depersonalize)

In [8]:
# df_fin.head()

In [9]:
# list(df_fin['text'])[1100:1200]

# IMPORT DVACH

In [10]:
import json
from tqdm import tqdm
import os
import random 

In [11]:
with open("C://input//toxic_and_unsafe//AGGREGATE_RAW_AND_PROCESSED_UNSAFE//arhivach_data//data//1.json") as f:
        data = json.load(f)

In [12]:
folder = "C://input//toxic_and_unsafe//AGGREGATE_RAW_AND_PROCESSED_UNSAFE//arhivach_data//data"

len(os.listdir(folder))

19674

In [13]:
collected_messages = []
for file in tqdm(os.listdir(folder)):
    path = os.path.join(folder,file)
    with open(path) as f:
        data = json.load(f)
        for mes in data:
            if len(mes['text'].split()) > 5 and len(mes['text']) < 250:
                collected_messages.append(mes['text'])

100%|████████████████████████████████████████████████████████████████████████████| 19674/19674 [08:16<00:00, 39.59it/s]


# IMPORT MAIL

In [14]:
df_mail = pd.read_csv("C://input//toxic_and_unsafe//AGGREGATE_RAW_AND_PROCESSED_UNSAFE//mail//qa_csv_from_answers_jsonl.csv")

C:\Users\N.Babakov\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [15]:
questions = set(df_mail['question'])

In [16]:
mail_collected = list(questions) + list(df_mail['answer'])

# IMPORT PIKABU

# Смешиваем и предобрабатываем

In [17]:
SAMPLE_NUMBER = 200000

In [18]:
mail_collected = random.sample(mail_collected, SAMPLE_NUMBER)

In [19]:
collected_messages = random.sample(collected_messages, SAMPLE_NUMBER)

In [20]:
# sources = ['mail'] * SAMPLE_NUMBER + ['dvach'] * SAMPLE_NUMBER

In [21]:
all_texts = collected_messages + mail_collected
del collected_messages
del mail_collected

In [36]:
all_texts = [s  for s in all_texts if isinstance(s,str)]

In [22]:
# df_all_samples = pd.DataFrame({'text':all_texts,"source":sources})
# df_all_samples.info()

In [26]:
import time
time.time()

1609666665.7204766

In [27]:
def clean_text(text):
    t = time.time()
    processed_text = depersonalize(text)
    print("depersonalize",time.time()-t)
    t = time.time()
    is_len_ok = get_len(processed_text)
    print("is_len_ok",time.time()-t)
    t = time.time()
    if is_len_ok is None:
        return None
#     print("is_mostly_russian...")
    is_rus = is_mostly_russian(processed_text)
    print("is_mostly_russian",time.time()-t)
    t = time.time()
    if is_rus is None:
        return None
#     print("existing_samples_set...")
    if processed_text in existing_samples_set:
        return None
    
    return processed_text

In [28]:
clean_text("фыыва привт332")

depersonalize 0.0
is_len_ok 0.0


In [47]:
depersonalize("фыыва привт332")

'фыыва привт'

In [37]:
# preprocessed_no_duplicates_list = []
# exception_count = 0
# for t in tqdm(all_texts):
#     if isinstance(t, str):
#         try:
#             cleaned_t = clean_text(t)
#             if cleaned_t is not None:
#                 preprocessed_no_duplicates_list.append(cleaned_t)
#         except:
#             exception_count +=1
#     if len(preprocessed_no_duplicates_list)%10000 and len(preprocessed_no_duplicates_list) > 0:
#         df_all_samples = pd.DataFrame({'text':preprocessed_no_duplicates_list})
#         df_all_samples.to_csv("OtvetiVS2ch_preprocessed_iteration2.csv", index = None)
# print("exception_count {}".format(exception_count))

In [30]:
tqdm.pandas()

C:\Users\N.Babakov\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\tqdm\std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


# apply

In [65]:
def is_in_existing_df(text):
    if text in existing_samples_set:
        return None
    return False

In [ ]:
# def clean_dataframe(df, text_column):
#     df.dropna(inplace = True)
#     df['processed'] = df[text_column].apply(depersonalize)
    
#     df['most_rus'] = df['processed'].apply(is_mostly_russian)
#     df.dropna(inplace = True)
    
#     df['good_len'] = df['processed'].apply(get_len)
#     df.dropna(inplace = True)
    
#     df['is_existent'] = df['processed'].apply(is_in_existing_df)
#     df.dropna(inplace = True)
    
#     del df_all_samples['most_rus']
#     del df_all_samples['good_len']
#     del df_all_samples['text']
#     del df_all_samples['is_existent']
    
#     return df

In [73]:
# df_pr = pd.DataFrame({"text":list(set(all_texts))})
df_pr = pd.DataFrame({"text":list(set(all_texts))})

In [40]:
len(df_pr)

395960

In [59]:
df_pr['processed'] =  df_pr['text'].progress_apply(depersonalize)
# df_pr['processed'] =  df_pr['text'].apply(depersonalize)
df_pr.head()

100%|█████████████████████████████████████████████████████████████████████████| 395960/395960 [40:52<00:00, 161.46it/s]


,text,processed
0,"попробуйте переехать на восток, недалеко, райо...","попробуйте переехать на восток, недалеко, райо..."
1,\n>>2896999И сколько времени должно пройти с н...,И сколько времени должно пройти с начала трени...
2,"ПОЛЬЗЫ ДЛЯ ОБЫЧНОГО ДВУНОГО НЕТ, ДЛЯ АНДРОДА С...","ПОЛЬЗЫ ДЛЯ ОБЫЧНОГО ДВУНОГО НЕТ, ДЛЯ АНДРОДА С..."
3,Могу предложить сталкера Battlefield Call of duty,Могу предложить сталкера Bttfd C f dty
4,да уж точно не то-что они там сейчас готовят.....,да уж точно не то-что они там сейчас готовят.....


In [93]:
df_pr = pd.read_csv("../merge_ready/SELF_MARKED_export_ready_st2.csv")
df_pr.head()

,processed,topic_1,topic_2
0,…Опять завыла собака. Я решился подойти к окну...,weapons,NaN
1,В правой руке она держала странный изогнутый и...,weapons,NaN
2,-проникновения инородных предметов в тело. ост...,weapons,NaN
3,"Так! быстро! У тебя по левой стороне раны, у м...",terrorism,NaN
4,"Моральной поддержки,чтоб в петлю не лезть...мы...",suicide,NaN


In [94]:
len(df_pr)

7499

In [95]:
df_pr['most_rus'] = df_pr['processed'].progress_apply(is_mostly_russian)

100%|███████████████████████████████████████████████████████████████████████████| 7499/7499 [00:00<00:00, 33012.12it/s]


In [97]:
Counter(df_pr['most_rus'])

Counter({True: 7498, None: 1})

In [98]:
df_pr.head()

,processed,topic_1,topic_2,most_rus
0,…Опять завыла собака. Я решился подойти к окну...,weapons,NaN,True
1,В правой руке она держала странный изогнутый и...,weapons,NaN,True
2,-проникновения инородных предметов в тело. ост...,weapons,NaN,True
3,"Так! быстро! У тебя по левой стороне раны, у м...",terrorism,NaN,True
4,"Моральной поддержки,чтоб в петлю не лезть...мы...",suicide,NaN,True


In [99]:
df_pr.dropna(inplace = True)
df_pr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33 entries, 14 to 4284
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   processed  33 non-null     object
 1   topic_1    33 non-null     object
 2   topic_2    33 non-null     object
 3   most_rus   33 non-null     object
dtypes: object(4)
memory usage: 1.3+ KB


In [86]:
df_pr['good_len'] = df_pr['processed'].apply(get_len)
df_pr.dropna(inplace = True)
df_pr.head()

,processed,topic_1,topic_2,most_rus,good_len
14,"хочу в прокуратуру подать заявление,доведен...",suicide,crime_real,True,True
36,Если всё таки пойду то потрачу на психотерапев...,suicide,halth_shaming,True,True
78,отчего-то вспомнила эту тему... 2 алко-коктей...,suicide,drugs,True,True
157,Пока мы не получили абстрактное мышление и не ...,suicide,drugs,True,True
1354,А в суд коллекторы как с монитором пойдут?Пони...,social,crime_web,True,True


In [87]:
df_pr['is_existent'] = df_pr['processed'].progress_apply(is_in_existing_df)

100%|████████████████████████████████████████████████████████████████████████████████| 33/33 [00:00<00:00, 4721.22it/s]


In [88]:
from collections import Counter
Counter(df_pr['is_existent'])

Counter({False: 33})

In [80]:
df_pr.dropna(inplace = True)
df_pr.head()

,text,processed,most_rus,good_len,is_existent
1,\n>>2896999И сколько времени должно пройти с н...,И сколько времени должно пройти с начала трени...,True,True,False
2,"ПОЛЬЗЫ ДЛЯ ОБЫЧНОГО ДВУНОГО НЕТ, ДЛЯ АНДРОДА С...","ПОЛЬЗЫ ДЛЯ ОБЫЧНОГО ДВУНОГО НЕТ, ДЛЯ АНДРОДА С...",True,True,False
3,Могу предложить сталкера Battlefield Call of duty,Могу предложить сталкера Bttfd C f dty,True,True,False
4,да уж точно не то-что они там сейчас готовят.....,да уж точно не то-что они там сейчас готовят.....,True,True,False
5,"Фига се, жизнь животных губить за яйца, а мясо...","Фига се, жизнь животных губить за яйца, а мясо...",True,True,False


In [81]:
del df_pr['text']
del df_pr['most_rus']
del df_pr['is_existent']
del df_pr['good_len']

In [82]:
len(df_pr)

315143

In [72]:
# df_pr.to_csv("OtvetiVS2ch_preprocessed-NO_FILTER.csv", index = None) 
df_pr.to_csv("OtvetiVS2ch_preprocessed_iteration2.csv", index = None)